In [15]:
import time
import pandas as pd
import numpy as np
import os
import cv2 as cv
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,\
                                    Conv2D,\
                                    MaxPooling2D,\
                                    Dropout,\
                                    Flatten
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,\
                                       ModelCheckpoint
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [3]:
# data loading
data = pd.DataFrame(columns=['Image', 'Class', 'FileName'])

for data_dir in sorted(os.listdir(r"/media/romes_papa/4116c49e-4877-48e1-938d-e7fb6cc948ea/romes_papa/Datasets/license_plates/Nums_data")):
    for file in sorted(os.listdir(fr"/media/romes_papa/4116c49e-4877-48e1-938d-e7fb6cc948ea/romes_papa/Datasets/license_plates/Nums_data/{data_dir}")):
        
        img_path = fr"/media/romes_papa/4116c49e-4877-48e1-938d-e7fb6cc948ea/romes_papa/Datasets/license_plates/Nums_data/{data_dir}/{file}"
        img = cv.imread(img_path, cv.IMREAD_COLOR)
        
        data = pd.concat([
            data,
            pd.DataFrame(data=[[img, data_dir, file]],
                         columns=['Image', 'Class', 'FileName'])
        ])

data.index = pd.RangeIndex(data.shape[0])

data['Height'] = data.Image.apply(lambda img: img.shape[0])
data['Width']  = data.Image.apply(lambda img: img.shape[1])

data.replace(to_replace = {'Class': {'Negative': 0, 'NumBase': 1}}, inplace=True, regex = True)

data

,Image,Class,FileName,Height,Width
0,"[[[69, 73, 78], [73, 77, 82], [74, 78, 83], [7...",0,0.bmp,144,264
1,"[[[116, 117, 121], [116, 117, 121], [116, 115,...",0,1.bmp,76,220
2,"[[[57, 58, 56], [57, 58, 56], [64, 65, 63], [7...",0,10.bmp,32,456
3,"[[[138, 150, 152], [166, 180, 192], [125, 144,...",0,100.bmp,114,158
4,"[[[220, 223, 238], [209, 213, 231], [194, 200,...",0,1000.bmp,178,318
...,...,...,...,...,...
6333,"[[[139, 110, 190], [136, 107, 186], [139, 112,...",1,sample_picture_2014-02-17_18-39-00.jpg,103,310
6334,"[[[33, 20, 36], [35, 22, 38], [36, 22, 40], [3...",1,sample_picture_2014-02-17_18-39-22.jpg,75,226
6335,"[[[33, 20, 36], [35, 22, 38], [36, 22, 40], [3...",1,sample_picture_2014-02-17_18-39-30.jpg,75,226
6336,"[[[67, 51, 104], [69, 53, 106], [72, 55, 112],...",1,sample_picture_2014-02-17_18-42-17.jpg,91,273


In [4]:
print(f'Average image height: {np.round(data.Height.mean(), 2)} px')
print(f'Average image  width: {np.round(data.Width .mean(), 2)} px')

Average image height: 130.92 px
Average image  width: 320.84 px


In [5]:
data.Image = data.Image.apply(lambda img: cv.resize(img, dsize=(320, 130)))

data['Height'] = data.Image.apply(lambda img: img.shape[0])
data['Width']  = data.Image.apply(lambda img: img.shape[1])

data

,Image,Class,FileName,Height,Width
0,"[[[69, 73, 78], [72, 76, 81], [73, 77, 82], [7...",0,0.bmp,130,320
1,"[[[116, 117, 121], [116, 117, 121], [116, 116,...",0,1.bmp,130,320
2,"[[[57, 58, 56], [61, 62, 60], [71, 75, 69], [7...",0,10.bmp,130,320
3,"[[[138, 150, 152], [145, 157, 161], [159, 172,...",0,100.bmp,130,320
4,"[[[221, 224, 239], [208, 211, 229], [191, 197,...",0,1000.bmp,130,320
...,...,...,...,...,...
6333,"[[[139, 110, 190], [136, 107, 186], [139, 112,...",1,sample_picture_2014-02-17_18-39-00.jpg,130,320
6334,"[[[33, 20, 36], [34, 21, 37], [35, 22, 38], [3...",1,sample_picture_2014-02-17_18-39-22.jpg,130,320
6335,"[[[33, 20, 36], [34, 21, 37], [35, 22, 38], [3...",1,sample_picture_2014-02-17_18-39-30.jpg,130,320
6336,"[[[67, 51, 104], [69, 52, 105], [71, 54, 110],...",1,sample_picture_2014-02-17_18-42-17.jpg,130,320


In [6]:
X = []
y = []

for image, target in zip(data.Image, data.Class):
    X.append(image)
    y.append(target)

X = np.array(X)
y = np.array(y)

y = utils.to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.1, random_state=228)

In [8]:
print(f'X_train.shape = {X_train.shape}')
print(f'X_test.shape = {X_test.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'y_test.shape = {y_test.shape}')

X_train.shape = (5704, 130, 320, 3)
X_test.shape = (634, 130, 320, 3)
y_train.shape = (5704, 2)
y_test.shape = (634, 2)


In [9]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.01,
    width_shift_range=0.01,
    height_shift_range=0.01
)

In [10]:
# model creating
num_classes = data.Class.unique().shape[0]

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                 activation='relu', input_shape=(130, 320, 3)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                 activation='relu'))
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                 activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                 activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 130, 320, 32)      2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 130, 320, 32)      25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 65, 160, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 65, 160, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 65, 160, 32)       25632     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 65, 160, 32)       25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 80, 32)        0

In [11]:
checkpoint = ModelCheckpoint('micro_detector.hdf5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1)

In [12]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

In [13]:
batch_size = 40
np.unique(y, axis=1).shape

(6338, 2)

In [16]:
start_time = time.time()
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    epochs=30,
                    validation_data=(X_test, y_test),
                    steps_per_epoch=(X_train.shape[0] // batch_size),
                    verbose=1,
                    callbacks=[checkpoint, learning_rate_reduction]
                    )
print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/30
142/142 [==============================] - ETA: 0s - loss: 0.9770 - accuracy: 0.8667
Epoch 00001: val_accuracy improved from -inf to 0.96530, saving model to micro_detector.hdf5
142/142 [==============================] - 47s 332ms/step - loss: 0.9770 - accuracy: 0.8667 - val_loss: 0.2083 - val_accuracy: 0.9653 - lr: 0.0010
Epoch 2/30
142/142 [==============================] - ETA: 0s - loss: 0.1699 - accuracy: 0.9500
Epoch 00002: val_accuracy improved from 0.96530 to 0.97792, saving model to micro_detector.hdf5
142/142 [==============================] - 38s 266ms/step - loss: 0.1699 - accuracy: 0.9500 - val_loss: 0.1117 - val_accuracy: 0.9779 - lr: 0.0010
Epoch 3/30
142/142 [==============================] - ETA: 0s - loss: 0.1262 - accuracy: 0.9629
Epoch 00003: val_accuracy did not improve from 0.97792
142/142 [==============================] - 36s 253ms/step - loss: 0.1262 - accuracy: 0.9629 - val_loss: 0.0921 - val_accuracy: 0.9779 - lr: 0.0010
Epoch 4/30
142/142 [=======

Epoch 27/30
142/142 [==============================] - ETA: 0s - loss: 0.0109 - accuracy: 0.9966
Epoch 00027: val_accuracy did not improve from 0.99527

Epoch 00027: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
142/142 [==============================] - 42s 298ms/step - loss: 0.0109 - accuracy: 0.9966 - val_loss: 0.0215 - val_accuracy: 0.9905 - lr: 6.2500e-05
Epoch 28/30
142/142 [==============================] - ETA: 0s - loss: 0.0113 - accuracy: 0.9956
Epoch 00028: val_accuracy did not improve from 0.99527
142/142 [==============================] - 42s 296ms/step - loss: 0.0113 - accuracy: 0.9956 - val_loss: 0.0232 - val_accuracy: 0.9905 - lr: 3.1250e-05
Epoch 29/30
142/142 [==============================] - ETA: 0s - loss: 0.0134 - accuracy: 0.9958
Epoch 00029: val_accuracy did not improve from 0.99527
142/142 [==============================] - 42s 296ms/step - loss: 0.0134 - accuracy: 0.9958 - val_loss: 0.0218 - val_accuracy: 0.9905 - lr: 3.1250e-05
Epoch 30/3